## Energy Innovation MCOE Compilation

- <a href=#setup>Setup</a>
- <a href=#data_out>Data Outputs</a>
    * <a href=#final-plant>Plant Level Output</a>
    * <a href=#final-unit>Unit Level Output</a>
    * <a href=#export>Export to CSV</a>
- <a href=#data_comp>Data Components</a>
    * <a href=#part1>Part 1: Basic Plant & Unit Information</a>
    * <a href=#part2>Part 2: Cost Data</a>
    * <a href=#part3>Part 3: Emissions & Public Health Data</a>
- <a href=#data_val>Data Validation</a>
    * <a href=#ferc-v-eia>FERC Form 1 vs. EIA</a>
    * <a href=#mcoe-validate>MCOE Validation</a>

-------------

## <a id='setup'>Setup</a>

In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import pudl
import sqlalchemy as sa
from ei_mcoe import *
import sys
import os
import matplotlib.pyplot as plt
import logging
from datetime import datetime

from copy import deepcopy

In [3]:
# basic setup for logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None

In [4]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])

In [5]:
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,
    start_date='2019-01-01',
    freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=False,
)

## <a id='data_out'>Data Outputs</a>

In [6]:
%%time
cems_df = get_cems(pudl_settings, years=[2019, 2020])

Getting CEMS data....this may take a sec.


/Users/christinagosnell/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60566 instead
  warnings.warn(


CPU times: user 4.3 s, sys: 1.2 s, total: 5.5 s
Wall time: 42.1 s


### <a id='final-plant'>Plant Level Output</a>

In [7]:
%%time
# beware - this takes several minutes run!!
mcoe_plant = main(pudl_out, cems_df, 'plant-fuel', separate_nems_cols=True, fill_in_fuel_cost=True)

Prepping raw EIA data
filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages


/Users/christinagosnell/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/pandas/core/reshape/merge.py:784: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  result[self.indicator_name] = Categorical(


Converting date to year
Calculating generator age
Converting date to year
Building Part 1 output


/Users/christinagosnell/code/ei-mcoe-map/ei_mcoe.py:782: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['_data_times_weight'] = df[data_col] * df[weight_col]
/Users/christinagosnell/code/ei-mcoe-map/ei_mcoe.py:783: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['_weight_where_notnull'] = df.loc[df[data_col].notnull(), weight_col]


Finished compiling Part 1 plant-fuel level data
Cleaning Part 1 Output
Finished cleaning Part 1 data
Building Part 3 output
 - Adding cems to EIA data
 - Adding pm values to the dataframe
 -- Calculating tech percents
 -- Calculating pm2.5 values
 - Calculating public health damages from emissions


/Users/christinagosnell/code/ei-mcoe-map/ei_mcoe.py:2041: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pm_df.assign(


 - Calculating the amount of premature deaths
Finished building Part 3 plant-fuel level data
Cleaning Part 3 output
Finished cleaning Part 3 output
Building Part 2 output
 - Building eia table broken down by plant and fuel type
 - Building FERC table broken down by plant
 - Readying EIA fuel pct data to merge with FERC
 - Building eia table broken down by plant and fuel type
 -- Calculating eia fuel type percentages
Turning eia fuel percent values for net_generation_mwh into columns
Turning eia fuel percent values for capacity_mw into columns
 - Merging FERC Form 1 data with EIA percent data
 -- Building FERC table broken down by plant and fuel type
 -- Melting FERC pct data back to row values
 -- Melting FERC pct data back to row values
 - Merging FERC and EIA mcoe data on plant and fuel type
 - Adding NEMS cost data
checking nems inside add_nems
Id's from NEMS:    5495
Ids w/ NEMS data:  0
Ids w/o NEMS data: 10130
Missing ids:       5273
checking nems inside build_part2_output
Id's f

/Users/christinagosnell/code/ei-mcoe-map/ei_mcoe.py:782: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['_data_times_weight'] = df[data_col] * df[weight_col]
/Users/christinagosnell/code/ei-mcoe-map/ei_mcoe.py:783: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['_weight_where_notnull'] = df.loc[df[data_col].notnull(), weight_col]


Finished compiling Part 1 unit-fuel level data
preparing merge: checking df length compatability
df passed not the same length as plant-level aggreation df;                    check for duplicates
Finished compiling Part 2 data compilation
Cleaning Part 2 output
inserting 80% fixed/var ratio for 10982 records
Calculate MCOE.
Finished cleaning Part 2 data
Assigned state FIPS codes for 99.96% of records.
Assigned county FIPS codes for 93.86% of records.
Finished compiling all parts!


In [23]:
mcoe_plant[(mcoe_plant.report_date.dt.year == NEMS_YEAR)
           & (mcoe_plant.fuel_type_code_pudl == 'coal')
          ].groupby(['confidence_tier'], dropna=False)[['plant_id_eia']].count()

,plant_id_eia
confidence_tier,
1.0,113
2.0,71
3.0,33
4.0,3
5.0,11
6.0,26
NaN,34


### <a id='final-unit'>Unit Level Output</a>

In [ ]:
#unit = main(pudl_out, cems_df, 'unit-fuel', separate_nems_cols=True)

### <a id='export'>Export to CSV</a>

In [19]:
mcoe_plant_19 = mcoe_plant[mcoe_plant.report_year == NEMS_YEAR]
source_df = generate_source_df()

mcoe_plant_19.to_csv(f"mcoe_compilation_2019-{datetime.today().strftime('%Y-%m-%d')}.csv",index=False)
source_df.to_csv('mcoe_documentation.csv',index=False)

Generating separate source dataframe


NameError: name 'datetime' is not defined

-------------------------

## <a id='data_comp'>Data Components</a>

### <a id='part1'>Part 1: Plant & Unit Level Data</a>
EIA-860 and EIA-923 generator-level data is aggregated by either plant or unit and subdivided by broad fuel type (coal, gas, oil, waste). 
[`Age`] is calculated by finding the weighted average (capacity as weight) of the generators in a given aggregation. 
[`Heat Rate`] is also calculated by weighted average (this time with net generation as weight). 
[`MW Nameplate Capacity`] and [`Annual Electricity Net Generation MWh`] at a given level of aggregation are calculated by summing generator-level data. 

For purely qualitative information (just plant name and location) add [`drop_calcs=True`] to the parameters.

### <a id='part2'>Part 2: Cost Data</a>
Cost and generation data from EIA-860, EIA-923, and FERC Form 1 are subdivided by plant and broad fuel type. The fuel-type breakdown for FERC Form 1 plants is determined by the EIA fuel breakdown for plants of the same PUDL Plant Code. For missing fixed and variable costs from 2018, we've input data from NEMS as a subsitutde. MCOE is calculated using data from the following sources:

##### NEMS Variable Origins
- net_generation_mwh_nems = capacity_factor * 8760 * capacity_mw
- fixed_om_18_nems = fixed_om_kw_18_nems * 1000 * capacity_mw
- fixed_om_mwh_18_nems = fixed_om_18_nems / net_generation_mwh_nems
- variable_om_18_nems = variable_om_mwh_18_nems * net_generation_mwh_nems
- fix_var_om_mwh_18_nems = variable_om_mwh_18_nems + fixed_om_kw_18_nems
- fixed_v_total_ratio = fixed_om_18_nems / (fixed_om_18_nems + variable_om_18_nems)
- var_v_total_ratio = variable_om_18_nems / (fixed_om_18_nems + variable_om_18_nems)
- fix_var_om_18_nems =  fixed_om_18_nems + variable_om_18_nems
- fix_var_om_mwh_18_nems = fixed_om_mwh_18_nems + variable_om_mwh_18_nems

##### MCOE Variable Origins 
- total_fuel_cost (Fuel cost) **EIA-923**
- capacity_mw (MW Capacity) **EIA-860**
- net_generation_mwh (Net MWh Generated) **EIA-923**: 
- opex_nofuel_ferc1 (Non-Fuel O&M) = **FERC Form 1**: opex_production_total - opex_fuel
- fixed_om = fix_var_om * fixed_v_total_ratio
- variable_om = fix_var_om * var_v_total_ratio
- fixed_om_mwh = fixed_om / net_generation_mwh_ferc1; if null, filled in with fixed_om_mwh_18_nems
- fixed_om_mw = fixed_om / capacity_mw
- variable_om_mwh = variable_om / net_generation_mwh_ferc1; if null, filled in with variable_om_mwh_18_nems
- fix_var_om_mwh = opex_nofuel_ferc1 / net_generation_mwh_ferc1


##### Data Flags
[`Significant Heat Rate Discrepancy`] - a field indicating whether a plant fuel type contains units that have outlier heatrates. If a unit is more than one standard deviation away from the mean value for units of its same fuel type rate, the field will appear [`True`].

[`Fixed/Variable O&M used NEMS?`] - a field indicating whether the given row used FERC Form 1 cost data or NEMS cost data. If NEMS were used, the field will appear [`True`].


### <a id='part3'>Part 3: Emissions & Public Health Data</a>

CEMS, or Continuous Emission Monitoring Systems, provide detailed information about gas, particulate matter and other pollutants that emanate from various point sources. Here, CEMS data on co2, so2, and nox emissions from generation units is combined with EIA plant data at the plant and unit level, separated by fuel type. 

Data on PM2.5 emissions comes from Argonne National Laboratory's GREET Model. The model's Electricity Generation Module table 2.1 contains PM2.5 emissions data in g/kwh at the grandularity of NERC region and technology type. The PM2.5 emissions data are mapped onto EIA and CEMS data by creating buckets of the same granularity.


--------------

## <a id='data_val'>Data Validation</a>

### <a id='ferc-v-eia'>FERC Form 1 vs. EIA</a>

The first test looks at the **validity of using EIA fuel percentage values to disaggregate FERC Form 1 data by fuel type.** 


The following hisograms compare the fuel fractions available in FERC Form 1 with the fuel fractions created by aggregating EIA data by plant and fuel type. The cost factors used in the calculation of MCOE rely on FERC Form 1 fixed and operating cost data broken down by plant and fuel type based on EIA fuel breakdown. To ensure that there is a degree of similarity between the percent breakdown of EIA fuel break down and FERC Form 1 breakdown, this histogram depicts the EIA percent / FERC Form 1 percent. FERC Form 1 has two fuel breakdowns, by MMBtu and by cost. The graph to the left divides EIA percents by FERC Form 1 MMBtu fuel fractions and on the right by cost fractions. The closer the value to 1, the more acurate the comparison.

In [ ]:
#plot_fuel_pct_check(merge_ferc1_eia_fuel_pcts(pudl_out))

As we know, FERC Form 1 and EIA data don't always match up properly. The following graphs depict **the difference in FERC Form 1 and EIA-860/923 reporting on the these particular values:** [`capacity_mw`], [`opex_fuel`], [`total_mmbtu`], [`net_generation_mwh`], [`capacity_factor`], [`heat_rate_mmbtu_mwh`], [`fuel_cost_per_mwh`], [`fuel_cost_per_mmbtu`], used in the calculation of MCOE.

In [ ]:
plot_eia_v_ferc(pudl_out)

### <a id='heatrate'>Heat Rate Comparison</a>

In [ ]:
plot_heat_rate(pudl_out)

### <a id='mcoe-validate'>Check the MCOE Compilation</a>

this is a check to see the general shape of the fixed and variable O&M of the PUDL data is similar to NEMS

In [ ]:

plant_non_nems = plant[~plant[['fix_var_is_NEMS']].astype(pd.BooleanDtype()).fix_var_is_NEMS]
#plant_non_nems = plant_19[(~plant_19.fix_var_is_NEMS)]
plt.hist(plant_non_nems.fix_var_om_mwh, density=True, cumulative=True, 
         range=(0,100), label='PUDL (Non-Nems)', alpha=.5,
         bins=100);

plt.hist((plant.fix_var_om_mwh_19_nems), density=True, cumulative=True, 
         range=(0,100), label='NEMS', alpha=.7,
         bins=100);
plt.xlabel('Non-Fuel OM $/MWh')
plt.legend()
plt.title("PUDL vs NEMS Non-Fuel OM Distribution")
plt.show()

The following graph, we're simply plotting the components of MCOE against MCOE.
This should be just a straight line... which it appears to be, so yay.
The `mcoe` column has the least amount of records in it because it requires having both fuel cost data from EIA and fixed and variable cost data from FERC or NEMS.

In [ ]:
plot_mcoe_vs_nems(plant, #plant_19, 
                  x_cols=['mcoe'], y_cols=['fix_var_om_mwh','fuel_cost_mwh_eia923'], 
                  log=True, 
                  x_lim=(.1,1e5), y_lim=(.1,1e5),
                  alt_title='MCOE v MCOE components'
                 );

Now we are getting into comparing the components of MCOE. First let's compare the fixed and variable dollars per MWh.
Here are the plots

In [ ]:
plot_mcoe_vs_nems(plant, 
                  x_cols=['fix_var_om_mwh'], y_cols=['fix_var_om_mwh_19_nems'], 
                  log=True, 
                  x_lim=(1,1e3), y_lim=(1,1e3),
                  #fuels=['coal','gas']
                 );
plot_mcoe_vs_nems(plant_non_nems, 
                  x_cols=['fix_var_om_mwh'], y_cols=['fix_var_om_mwh_19_nems'], 
                  log=True, 
                  x_lim=(1,1e2), y_lim=(1,1e2),
                  #fuels=['coal','gas']
                 );

In [ ]:
plot_mcoe_vs_nems(plant_non_nems, 
                  x_cols=['fixed_om_mwh'], y_cols=['fixed_om_mwh_19_nems'], 
                  log=True, 
                  x_lim=(1,1e3),
                  y_lim=(1,1e3), 
                  #fuels=['coal','gas']
                 );
plot_mcoe_vs_nems(plant_non_nems, 
                  x_cols=['variable_om_mwh'], y_cols=['variable_om_mwh_19_nems'], 
                  log=True, 
                  x_lim=(.01,1e2),
                  y_lim=(.01,1e2)
                 );

There are the total fixed and variable dollar amounts. They look quite reasonable against NEMS totals.

In [ ]:
plot_mcoe_vs_nems(plant, 
                  x_cols=['fix_var_om'], y_cols=['fix_var_om_19_nems'], 
                  log=True, 
                  x_lim=(1e5,1e9),
                  y_lim=(1e5,1e9)
                 );
plot_mcoe_vs_nems(plant, 
                  x_cols=['fixed_om'], y_cols=['fixed_om_19_nems'], 
                  log=True, 
                  x_lim=(1e5,1e9),
                  y_lim=(1e5,1e9)
                 );
plot_mcoe_vs_nems(plant,
                  x_cols=['variable_om'], y_cols=['variable_om_19_nems'], 
                  log=True, 
                  x_lim=(1e5,1e9),
                  y_lim=(1e5,1e9)
                 );

In [ ]:
plot_hist_annual(plant_non_nems, 'mcoe', "MCOE $/MWh")

In [ ]:
plot_hist_annual(plant_filled, 'fix_var_om_mwh', "O&M $/MWh")

In [ ]:
# this is a brief exploration of the split between fixed and variable costs from NEMS.
# there are clear patterns/differences between coal and gas.
# right now, we aren't using fuel type averages to break out fixed and variable costs
# but we could for the records which don't have associated NEMS data

nems = prep_nems(pudl_out)
fuel_tpyes= nems.fuel_type_code_pudl.unique()
#for fuel_type in fuel_tpyes:
for fuel_type in ['gas','coal']:
    df = nems[nems.fuel_type_code_pudl == fuel_type]
    plt.hist(df.fixed_v_total_ratio, 
             range=(0,1),
             weights=df.capacity_mw, 
             label=f'{fuel_type}',
             bins=40
            )
    plt.title(f"Fixed v Variable Non-Fuel O&M in NEMS for {fuel_type}")
    plt.xlabel("Ratio")
    plt.ylabel("Capacity MW")
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
# quick exploration of mcoe
plant = plant_filled
plant_19 = plant[plant.report_year == 2019]
logger.info(f'all 2019 records:    {len(plant_19)}')
logger.info(f'2019 records w/ferc: {len(plant_19[plant_19.fix_var_om_mwh.notnull()])}')
logger.info(f'2019 records w/nems: {len(plant_19[plant_19.fix_var_om_mwh_19_nems.notnull()])}')
logger.info(f'2019 records w/eia:  {len(plant_19[plant_19.fuel_cost_mwh_eia923.notnull()])}')
logger.info(f'2019 records w/moce: {len(plant_19[plant_19.mcoe.notnull()])}')

In [ ]:
# quick exploration of capacity
plant_19 = plant[plant.report_year == 2019]
plant_19_coal_gas = plant_19[plant_19.fuel_type_code_pudl.isin(['coal','gas',])]
logger.info(f'all 2019 capacity:    {(plant_19_coal_gas.capacity_mw.sum())/plant_19_coal_gas.capacity_mw.sum():.01%}')
logger.info(f'2019 capacity w/ferc: {(plant_19_coal_gas[plant_19_coal_gas.fix_var_om_mwh.notnull()].capacity_mw.sum())/plant_19_coal_gas.capacity_mw.sum():.01%}')
logger.info(f'2019 capacity w/nems: {(plant_19_coal_gas[plant_19_coal_gas.fix_var_om_mwh_19_nems.notnull()].capacity_mw.sum())/plant_19_coal_gas.capacity_mw.sum():.01%}')
logger.info(f'2019 capacity w/eia:  {(plant_19_coal_gas[plant_19_coal_gas.fuel_cost_mwh_eia923.notnull()].capacity_mw.sum())/plant_19_coal_gas.capacity_mw.sum():.01%}')
logger.info(f'2019 capacity w/mcoe: {(plant_19_coal_gas[plant_19_coal_gas.mcoe.notnull()].capacity_mw.sum())/plant_19_coal_gas.capacity_mw.sum():.01%}')

In [ ]:
# capacity coverage
plant_19[plant_19.mcoe.notnull()].capacity_mw.sum()/plant_19[plant_19.fuel_type_code_pudl.isin(['coal','gas'])].capacity_mw.sum()

In [ ]:
# net generation coverage
plant_19[plant_19.mcoe.notnull()].net_generation_mwh.sum()/plant_19[plant_19.fuel_type_code_pudl.isin(['coal','gas'])].net_generation_mwh.sum()

In [ ]:
# coal capacity coverage
(plant_19[(plant_19.mcoe.notnull())
          & plant_19.fuel_type_code_pudl.isin(['coal'])
         ].capacity_mw.sum()
 /plant_19[(plant_19.fuel_type_code_pudl.isin(['coal']))].capacity_mw.sum())

In [ ]:
#id_pudl = 661
#id_pudl = 411 # NS
id_pudl = 1
cols_plant = ['plant_id_pudl', 'plant_id_eia', 'plant_name_eia', 'fuel_type_code_pudl', 'generator_id']

In [ ]:
#plant[plant.plant_id_pudl == id_pudl].set_index(cols_plant)

In [ ]:
plants_steam_ferc1 = pudl_out.plants_steam_ferc1()
g = pudl_out.gens_eia860()
m = pudl_out.mcoe()

In [ ]:
cols_fp = ['plant_id_pudl', 'plant_name_ferc1', 'utility_name_ferc1','report_year']

In [ ]:
ps[(ps.plant_id_pudl == id_pudl)
   & (ps.report_year == 2019)
  ].set_index(cols_fp)[
    ['net_generation_mwh', 'opex_fuel', 'opex_production_total', ]
].assign(opex_nofuel_ferc1=lambda x: (x.opex_production_total -
                                             x.opex_fuel),
                fix_var_om_mwh=lambda x: (
                    x.opex_nofuel_ferc1 / x.net_generation_mwh)
                )
#[['capacity_mw','plant_type','installation_year']]

In [ ]:
input_dict['plant_index_cols']

In [ ]:
mask_dolet_hills_2019 = (plants_steam_ferc1.plant_id_pudl == id_pudl) & (plants_steam_ferc1.report_year == 2019)

In [ ]:
(
    plants_steam_ferc1
    [mask_dolet_hills_2019]
    .groupby(['plant_id_pudl', 'report_year'], as_index=False)
    [['opex_fuel','opex_production_total','net_generation_mwh']]
    .sum(min_count=1)
    .assign(
        opex_nofuel_ferc1=lambda x: (
            x.opex_production_total - x.opex_fuel),
        fix_var_om_mwh=lambda x: (
                x.opex_nofuel_ferc1 / x.net_generation_mwh)
    )
)

In [ ]:
g[(g.plant_id_pudl == id_pudl)
  & (g.report_date.dt.year == 2019)
 ].set_index(cols_plant)[['capacity_mw','utility_name_eia','operating_date', 'prime_mover_code','operational_status', 'latitude', 'longitude']]
#.capacity_mw.sum()
#.report_date.dt.year.unique()

In [ ]:
g[(g.plant_id_pudl == id_pudl)
  #& (g.report_date.dt.year == 2019)
 ][cols_plant].drop_duplicates()

In [ ]:
g[(g.plant_id_eia == 55958)
  #& (g.report_date.dt.year == 2019)
 ]#[cols_plant].drop_duplicates()

In [ ]:
m[(m.plant_id_pudl == id_pudl) & (m.report_date.dt.year == 2019)]

In [ ]:
g[(g.plant_id_eia.isin([58378,2357])) & (g.report_date.dt.year == 2019)]

In [ ]:
g[(g.plant_name_eia.str.contains('Dolet Hills')) & (g.report_date.dt.year == 2019)].set_index(cols_plant)[['city', 'state', 'latitude', 'longitude', 'capacity_mw']]

In [ ]:
#ps[ps.plant_name_ferc1.str.contains('arish')]